In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from LoraHelpers import convert_model_to_lora_model, change_lora_alpha
from data_loader import load_dataset_from_list, load_json_data
from tqdm.notebook import tqdm
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW

model_path = "./llama-3.2-1b-inst"
epochs = 10

In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
convert_model_to_lora_model(model)
change_lora_alpha(model, 0.125)

In [3]:
Crterion = CrossEntropyLoss(reduction="mean")
optimizer = AdamW(model.parameters(), lr=1e-4)

In [4]:
data_samples = load_json_data()
data = load_dataset_from_list(data_samples)

In [5]:
a = model.forward(torch.tensor(data[0]["input_tokens"]).unsqueeze(0)).logits
# a = torch.softmax(a, dim=-1)
a = a.softmax(dim=-1)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [6]:
a.shape, torch.tensor(data[0]["output_tokens"]).unsqueeze(0).shape, data[0]["sep_token_index"]

(torch.Size([1, 49, 128256]), torch.Size([1, 49]), 10)

In [7]:
torch.tensor(data[0]["output_tokens"])

tensor([  3513,   6261,     26,   5127,    775,    374,   2736,   4529,     13,
           765,   9681,     25,   2570,   1395,  53246,    726,    518,    364,
         61887,   9339,  17453,  78342,    518,    364,     71,  36224,     88,
           518,    364,   1354,   5682,   1697,    518,    364,  34965,  45213,
          2844,  83528,   7063,   2695,  35376,    518,    364,   3022,   3502,
          7164,  58182,    663, 128009])

In [8]:
Crterion(a.squeeze()[10:], torch.tensor(data[0]["output_tokens"][10:]))

tensor(11.3790, grad_fn=<NllLossBackward0>)

In [9]:
tokenizer.decode(a.argmax(dim=-1)[0])

"#ach, be else is already taken.\n - Unknown: selfself yourselfself','selfgames okins','selfappinessy', 'authpirationational','motunderuted',car-wil','self',spiror']\n\n |\n"

In [10]:
train_losses = []
for epoch in tqdm(range(epochs)):
    pbar = tqdm(range(len(data)), leave=False)
    for i in pbar:
        optimizer.zero_grad()
        mask_idx = data[i]["sep_token_index"]
        outs = model.forward(torch.tensor(data[i]['input_tokens']).unsqueeze(0)).logits.softmax(dim=-1)
        sample_loss = Crterion(outs.squeeze()[mask_idx:], torch.tensor(data[i]["output_tokens"][mask_idx:]))
        train_losses.append(sample_loss.detach())
        sample_loss.backward()
        optimizer.step()
        del outs
        del sample_loss
        pbar.set_description("" if len(train_losses)==0 else f"loss : {train_losses[-1]:.3f}")
    change_lora_alpha(model, 0.125*2**epoch)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2508 [00:00<?, ?it/s]

  0%|          | 0/2508 [00:00<?, ?it/s]

  0%|          | 0/2508 [00:00<?, ?it/s]

  0%|          | 0/2508 [00:00<?, ?it/s]

In [15]:
train_losses = [float(item) for item in train_losses]
train_losses

[11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,
 11.0,

In [30]:
change_lora_alpha(model, 0.5)

In [3]:
s = tokenizer("You shall not pass | tags", return_tensors='pt')['input_ids']
a = model.generate(s)
"".join([tokenizer.decode(token) for token in a[0]])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/adi/miniconda3/envs/llm/lib/python3.12/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'<|begin_of_text|>You shall not pass | tags: #fashion #style #beauty #fashionista'